## Getting Started with prompt Template and Chat Prompt Template

#### Use PromptTemplate to create a template for a string prompt.
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [57]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
import os 

In [8]:
load_dotenv()

True

In [25]:
#model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)
#model = ChatGoogleGenerativeAI(model="gemini-pro",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.8)
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-002",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.8)



### A simple string based Prompt formatting

In [26]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
#print(prompt_template.__doc__)
prompt = prompt_template.format(adjective="sad", content="British Weather")

response_one = model.invoke(prompt)
print(f'Response one: {response_one.content}')

# Create the Chain
chain = prompt_template | model
response_two = chain.invoke({"adjective": "funny", "content": "cats"})
print(f"Response two: {response_two.content}")

Response one: Why did the British man bring a ladder to the beach?

Because he heard the forecast was partly cloudy.  He was hoping to reach the sunny part.

Response two: Why was the cat sitting on the computer?  

To keep an eye on the mouse!



### ChatPromptTemplate: The prompt to chat models is a list of chat messages.
Each chat message is associated with content, and an additional parameter called role. <br>
For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [23]:
from langchain_core.prompts import ChatPromptTemplate

In [34]:
'''
The conversation in the template is structured like this:

System Message: "You are a helpful AI bot. Your name is {name}."

This is used to give context to the AI assistant.
Role: "system" – provides configuration and context for the AI model.
Content: Introduces the bot as a helpful AI with a placeholder for its name.
Human Message: "Hello, how are you doing?"

The human starts the conversation with a greeting.
Role: "human" – represents the human user speaking to the AI.
AI Message: "I'm doing well, thanks!"

The AI replies to the human.
Role: "ai" – represents the AI’s response to the initial human message.
Human Message: "{user_input}"

The human follows up with another input that will be provided dynamically.
Role: "human" – this represents the next message in the conversation, and it contains 
a placeholder ({user_input}) that will be filled with a real question or input later.
'''
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
'''
The .format_messages() method is used to replace placeholders with real values.
 This helps create the actual content that will be sent to the model.
Parameters:
name="Bob": This fills in the {name} placeholder, so the AI will refer to itself as "Bob".
user_input="What is your name and what are you good at?": This fills in the {user_input}
 placeholder with the actual question the human asks.
'''
prompt = chat_template.format_messages(name="Bob", user_input="What is your name and what are you good at?")

formatted_prompt_text = " ".join([message.content for message in prompt])
print(f'The Final Prompt was: {formatted_prompt_text}')
response = model.invoke(prompt)
print('The response : ******************')
print(response.content)

The Final Prompt was: You are a helpful AI bot. Your name is Bob. Hello, how are you doing? I'm doing well, thanks! What is your name and what are you good at?
The response : ******************
My name is Bob.  I'm good at a variety of things, including:

* **Understanding and generating human-like text:** I can answer your questions, write different kinds of creative content, and translate languages.
* **Following instructions and completing tasks as instructed:**  Tell me what you need, and I'll do my best to help.
* **Accessing and processing information:** I can draw on a massive dataset to provide you with information on a wide range of topics.
* **Learning and adapting:**  I'm constantly learning and improving my abilities.

What can I help you with today?



### Various ways of formatting System/Human/AI prompts

In [36]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
prompt = chat_template.format_messages(text="I don't like eating tasty things")

response = model.invoke(prompt)
print(response.content)

I'm discovering new foods and having fun trying different flavors!



### Providing a Context along with the Prompt

In [46]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [47]:
print(model.invoke(prompt))

content="Hugging Face's `transformers` library, OpenAI's `openai` library, and Cohere's `cohere` library offer LLMs.\n" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-61ed59f9-b69e-4578-86ba-f87926c8db0b-0' usage_metadata={'input_tokens': 126, 'output_tokens': 35, 'total_tokens': 161}


### Langchain’s ```FewShotPromptTemplate``` caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.<br>

<b>Fixed Examples</b><br>
The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production. <br>

The basic components of the template are: - examples: A list of dictionary examples to include in the final prompt. - example_prompt: converts each example into 1 or more messages through its format_messages method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

In [48]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, 
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

In [49]:
# create a prompt example from above template

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [50]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [51]:
query = "What movie should I watch this evening?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What movie should I watch this evening?
AI: 


In [53]:
chain = few_shot_prompt_template | model

chain.invoke({"query": query}).content

"Anything but the one you'll inevitably fall asleep during.  Unless that's your goal, in which case, I recommend a nature documentary.\n"

```Summary```

1-FewShotPromptTemplate is used to generate prompts that include a few examples to show the model the expected format and tone.

2-It is composed of prefix (instructions), few-shot examples, and suffix (the current user query).

3-This prompt helps the model generate responses that align with the tone and style of the few-shot examples.

4-It is particularly useful in few-shot learning settings where you want the model to generate consistent and contextually relevant responses based on limited examples.

This kind of template can be extremely effective in helping the model understand the desired output style and personality without explicitly training the model on a new dataset.